In [3]:
!pip install geopy geopandas

In [4]:
from geopy.geocoders import Nominatim
import geopy
import pandas as pd
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
import time

In [5]:
df = pd.read_csv("data/processed/processed_municip_kWh.csv", encoding="utf-8", index_col=0)
df.head() 

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree)
0,Alberta,Acadia Valley,January,3.78,3.57,3.83,3.11,4.31,1.23
1,Alberta,Acadia Valley,February,4.69,4.78,4.96,4.35,5.87,2.23
2,Alberta,Acadia Valley,March,4.79,5.58,5.49,5.37,7.20,3.63
3,Alberta,Acadia Valley,April,4.09,5.80,5.34,5.96,8.43,4.96
4,Alberta,Acadia Valley,May,3.37,5.59,4.88,6.06,9.06,5.84


In [6]:
df_annual = df[df['Month'].str.contains("Annual", na=False)].copy()
df_annual.loc[:, 'address'] = df_annual.loc[:, 'Municipality'] + ', ' + df_annual.loc[:, 'Province']
df_annual.reset_index(inplace=True, drop=True)
df_annual.head() 

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree),address
0,Alberta,Acadia Valley,Annual,3.85,4.95,4.71,4.94,7.09,3.73,"Acadia Valley, Alberta"
1,Alberta,Acme,Annual,3.72,4.76,4.53,4.76,6.77,3.62,"Acme, Alberta"
2,Alberta,Airdrie,Annual,3.66,4.70,4.47,4.70,6.67,3.61,"Airdrie, Alberta"
3,Alberta,Alberta Beach,Annual,3.52,4.44,4.21,4.46,6.35,3.42,"Alberta Beach, Alberta"
4,Alberta,Alder Flats,Annual,3.47,4.41,4.18,4.42,6.28,3.44,"Alder Flats, Alberta"


In [7]:
address = df_annual.iloc[0]["address"]
address

'Acadia Valley, Alberta'

In [8]:
len(df_annual['address'].unique())

3506

In [9]:
# try running this block first to see if can create latitude and longtitude of one single location 
loc = Nominatim(user_agent="Geopy Library")
getLoc = loc.geocode(address)

print(getLoc.address)
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

Acadia Valley, Division No. 4, Alberta, Canada
Latitude =  51.1586763 

Longitude =  -110.210332


In [10]:
geolocator = geopy.Nominatim(user_agent="Geopy Library")
lat = []
lon = []

for i, address in enumerate(df_annual.address[:50]):
    try:
        getLoc = geolocator.geocode(address)
        lat.append(getLoc.latitude)
        lon.append(getLoc.longitude)
    except:
        getLoc = geolocator.geocode(address)
        lat.append(pd.NA)
        lon.append(pd.NA)

    if i % 100 == 0:
        print(f'Index {i} completed')
        print(lat)
        print(lon)


Index 0 completed
[51.1586763]
[-110.210332]


In [12]:
len(lat), len(lon)

(50, 50)

In [24]:
# then run this 

if 'latitude' not in df_annual.columns:
    df_annual['latitude'] = pd.NA
if 'longitude' not in df_annual.columns:
    df_annual['longitude'] = pd.NA

# Set up the geolocator with rate limiting
geolocator = geopy.Nominatim(user_agent="Geopy Library")
rate_limiter = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # adjust min_delay_seconds as needed

for index, row in df_annual.iterrows():
    
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):  # Check if geocoding is needed
        try:
            location = rate_limiter(row['address'])
            if location:
                df_annual.at[index, 'latitude'] = location.latitude
                df_annual.at[index, 'longitude'] = location.longitude
        except geopy.exc.GeocoderServiceError as e:
            print(f"Error geocoding {row['address']}: {e}")

print(df_annual.head())  # Show the first few rows to check

KeyboardInterrupt: 